In [9]:
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GINConv # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import scipy.sparse
import pandas as pd
import pathlib
import os
import json

from sklearn.model_selection import train_test_split

In [4]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [ ]:
current_path = pathlib.Path().absolute()
path = current_path.parent

In [7]:
# load a npz file
# the npz file is a sparse matrix
# as all the matrices are equal, we can use the first one is equal to all the others

adj_angry_path = path / 'angry_adj'
print(adj_angry_path)
for file in os.listdir(adj_angry_path)[:1]:
    file_path = adj_angry_path / file
    sparse_matrix = scipy.sparse.load_npz(file_path)
    sparse_matrix = sparse_matrix.todense()


c:\Users\felip\OneDrive\Documentos\GitHub\Facial-Emotion-Classification\angry_adj


In [8]:
angry_path = path / 'angry_dist'
disgusted_path = path / 'disgusted_dist'
happy_path = path / 'happy_dist'
neutral_path = path / 'neutral_dist'
sad_path = path / 'sad_dist'
surprised_path = path / 'surprised_dist'

path_list = [angry_path, disgusted_path, happy_path, neutral_path, sad_path, surprised_path]

In [10]:
def extrai_dist(path_list, dists_list, limit=0.1):
    for path in path_list:
        quant_files = len(os.listdir(path))
        count = 0
        for file in os.listdir(path):
            if (count / quant_files) > limit:
                break
            file_path = path / file
            count += 1
            with open(file_path, 'r') as f:
                data = json.load(f)
                dists_list[path_list.index(path)].append(data)

In [11]:
dists_surprised = []
dists_disgusted = []
dists_happy = []
dists_neutral = []
dists_sad = []
dists_angry = []

dists_list = [dists_angry, dists_disgusted, dists_happy, dists_neutral, dists_sad, dists_surprised]

extrai_dist(path_list, dists_list, 0.1)

print(len(dists_angry))
print(len(dists_disgusted))
print(len(dists_happy))
print(len(dists_neutral))
print(len(dists_sad))
print(len(dists_surprised))

446
162
810
687
742
402


In [12]:
sparse_matrix.shape

(468, 468)

In [14]:
# create a mtrix that is the adjancency matrix multiplied by the features
# the features are the distances between the nodes
# each line of the matrix is a node and the columns are the distances between the node and the other nodes
# for each point = 1 in the adjacency matrix, multiply by the distances
# the result is a matrix with the distances between the nodes, which is the features matrix

def create_features_matrix(adj_matrix, dists_list):
    features_matrix = np.zeros((adj_matrix.shape[0], adj_matrix.shape[0]))
    for i in range(adj_matrix.shape[0]):
        for j in range(adj_matrix.shape[0]):
            if adj_matrix[i, j] == 1:
                #print(dists_list[i])
                features_matrix[i, j] = dists_list[i]
    return features_matrix

features_matrix = create_features_matrix(sparse_matrix, dists_list[0][0])
print(features_matrix.shape)
print(features_matrix)

(468, 468)
[[ 0.         67.1576676   0.         ...  0.          0.
   0.        ]
 [45.88164954  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [15]:
angry_features_matrix_list = []
for features_matrix in dists_angry:
    angry_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(angry_features_matrix_list))

446


In [16]:
disgusted_features_matrix_list = []
for features_matrix in dists_disgusted:
    disgusted_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(disgusted_features_matrix_list))

162


In [17]:
happy_features_matrix_list = []
for features_matrix in dists_happy:
    happy_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(happy_features_matrix_list))

810


In [18]:
neutral_features_matrix_list = []
for features_matrix in dists_neutral:
    neutral_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(neutral_features_matrix_list))

687


In [19]:
sad_features_matrix_list = []
for features_matrix in dists_sad:
    sad_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(sad_features_matrix_list))

742


In [20]:
surprised_features_matrix_list = []
for features_matrix in dists_surprised:
    surprised_features_matrix_list.append(create_features_matrix(sparse_matrix, features_matrix))

print(len(surprised_features_matrix_list))

402


In [21]:
# create a dataset with the features and target
# the target is an emotion
# features are feature matrices

# create a dataset (n,n) with the features matrix of each emotion
# in this dataset each colum is an element of the features matrix of each emotion

df_angry = pd.DataFrame()
for i in range(len(angry_features_matrix_list)):
    for j in range(angry_features_matrix_list[i].shape[0]):
        df_angry['node_' + str(j)] = angry_features_matrix_list[i][j]

df_disgusted = pd.DataFrame()
for i in range(len(disgusted_features_matrix_list)):
    for j in range(disgusted_features_matrix_list[i].shape[0]):
        df_disgusted['node_' + str(j)] = disgusted_features_matrix_list[i][j]

df_happy = pd.DataFrame()
for i in range(len(happy_features_matrix_list)):
    for j in range(happy_features_matrix_list[i].shape[0]):
        df_happy['node_' + str(j)] = happy_features_matrix_list[i][j]

df_neutral = pd.DataFrame()
for i in range(len(neutral_features_matrix_list)):
    for j in range(neutral_features_matrix_list[i].shape[0]):
        df_neutral['node_' + str(j)] = neutral_features_matrix_list[i][j]

df_sad = pd.DataFrame()
for i in range(len(sad_features_matrix_list)):
    for j in range(sad_features_matrix_list[i].shape[0]):
        df_sad['node_' + str(j)] = sad_features_matrix_list[i][j]

df_surprised = pd.DataFrame()
for i in range(len(surprised_features_matrix_list)):
    for j in range(surprised_features_matrix_list[i].shape[0]):
        df_surprised['node_' + str(j)] = surprised_features_matrix_list[i][j]

# add the target columns
df_angry['target'] = 0
df_disgusted['target'] = 1
df_happy['target'] = 2
df_neutral['target'] = 3
df_sad['target'] = 4
df_surprised['target'] = 5

# concat all the dataframes
df = pd.concat([df_angry, df_disgusted, df_happy, df_neutral, df_sad, df_surprised])

# shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

C:\Users\felip\AppData\Local\Temp\ipykernel_110472\3649290034.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_angry['node_' + str(j)] = angry_features_matrix_list[i][j]
C:\Users\felip\AppData\Local\Temp\ipykernel_110472\3649290034.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_angry['node_' + str(j)] = angry_features_matrix_list[i][j]
C:\Users\felip\AppData\Local\Temp\ipykernel_110472\3649290034.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many 

In [22]:
df.head()

,node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,node_9,...,node_459,node_460,node_461,node_462,node_463,node_464,node_465,node_466,node_467,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [23]:
df.shape

(2808, 469)

In [24]:
df['target'].value_counts()

1    468
0    468
5    468
4    468
2    468
3    468
Name: target, dtype: int64

In [25]:
# verify if there is any null value
df.isnull().sum()

node_0      0
node_1      0
node_2      0
node_3      0
node_4      0
           ..
node_464    0
node_465    0
node_466    0
node_467    0
target      0
Length: 469, dtype: int64

In [26]:
n_out = 6  # Number of classes
N = df.shape[0]  # Number of nodes in the graph
F = df.shape[1] - 1  # Original feature dimensionality

# GINConv Model

https://graphneural.network/layers/convolution/#ginconv

In [27]:
X_in = Input(shape=(N, F))
A_in = Input(tensor=sp_matrix_to_sp_tensor(sparse_matrix)) # A as a fixed tensor, otherwise Keras will complain about inputs of different rank.

In [28]:
graph_conv_1 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([X_in, A_in])
graph_conv_2 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([graph_conv_1, A_in])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

In [29]:
# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2808, 468)]          0         []                            
                                                                                                  
 input_2 (InputLayer)        [(468, 468)]                 0         []                            
                                                                                                  
 gin_conv (GINConv)          (None, 2808, 32)             15009     ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 gin_conv_1 (GINConv)        (None, 2808, 32)             1057      ['gin_conv[0][0]',        

In [31]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.33,
                                                    random_state=42)

X_train

,node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,node_9,...,node_458,node_459,node_460,node_461,node_462,node_463,node_464,node_465,node_466,node_467
2208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.164743,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,260.231531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Still need to figure out how to use the GINConv model

In [33]:
X_train_np = X_train.values
A_train_np = sp_matrix_to_sp_tensor(sparse_matrix)
y_train_np = y_train.values

model.fit(
    [X_train_np, A_train_np],
    y_train_np,
    batch_size=batch_size,
    validation_data=validation_data,
    epochs=epochs,
    callbacks=[
        EarlyStopping(patience=es_patience, restore_best_weights=True)
    ],
    verbose=1
)

ValueError: Dimensions 1881 and 468 are not compatible